In [1]:
import os
# Find the latest version of spark 3.0  from http://www-us.apache.org/dist/spark/ and enter as the spark version
# For example:
# spark_version = 'spark-3.0.2'
spark_version = 'spark-3.1.1'
os.environ['SPARK_VERSION']=spark_version

# Install Spark and Java
!apt-get update
!apt-get install openjdk-11-jdk-headless -qq > /dev/null
!wget -q http://www-us.apache.org/dist/spark/$SPARK_VERSION/$SPARK_VERSION-bin-hadoop2.7.tgz
!tar xf $SPARK_VERSION-bin-hadoop2.7.tgz
!pip install -q findspark

# Set Environment Variables
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-11-openjdk-amd64"
os.environ["SPARK_HOME"] = f"/content/{spark_version}-bin-hadoop2.7"

# Start a SparkSession
import findspark
findspark.init()

Get:1 https://cloud.r-project.org/bin/linux/ubuntu bionic-cran40/ InRelease [3,626 B]
Ign:2 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  InRelease
Get:3 http://ppa.launchpad.net/c2d4u.team/c2d4u4.0+/ubuntu bionic InRelease [15.9 kB]
Get:4 http://security.ubuntu.com/ubuntu bionic-security InRelease [88.7 kB]
Hit:5 http://archive.ubuntu.com/ubuntu bionic InRelease
Ign:6 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  InRelease
Get:7 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  Release [697 B]
Hit:8 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  Release
Get:9 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  Release.gpg [836 B]
Get:10 http://archive.ubuntu.com/ubuntu bionic-updates InRelease [88.7 kB]
Hit:11 http://ppa.launchpad.net/cran/libgit2/ubuntu bionic InRelease
Get:12 http://archive.ubuntu.com/ubuntu bionic-backpor

In [2]:
!wget https://jdbc.postgresql.org/download/postgresql-42.2.9.jar

--2021-04-21 15:23:52--  https://jdbc.postgresql.org/download/postgresql-42.2.9.jar
Resolving jdbc.postgresql.org (jdbc.postgresql.org)... 72.32.157.228, 2001:4800:3e1:1::228
Connecting to jdbc.postgresql.org (jdbc.postgresql.org)|72.32.157.228|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 914037 (893K) [application/java-archive]
Saving to: ‘postgresql-42.2.9.jar’

postgresql-42.2.9.j 100%[===================>] 892.61K  5.22MB/s    in 0.2s    

2021-04-21 15:23:52 (5.22 MB/s) - ‘postgresql-42.2.9.jar’ saved [914037/914037]



In [3]:
from pyspark.sql import SparkSession
spark = SparkSession.builder.appName("CloudETL").config("spark.driver.extraClassPath","/content/postgresql-42.2.9.jar").getOrCreate()

In [4]:
from pyspark import SparkFiles
# Load in employee.csv from S3 into a DataFrame
url = "https://s3.amazonaws.com/amazon-reviews-pds/tsv/amazon_reviews_us_Automotive_v1_00.tsv.gz"
spark.sparkContext.addFile(url)

df = spark.read.option('header', 'true').csv(SparkFiles.get("amazon_reviews_us_Automotive_v1_00.tsv.gz"), inferSchema=True, sep='\t', timestampFormat="yyyy-mm-dd")
df.show(10)

+-----------+-----------+--------------+----------+--------------+--------------------+----------------+-----------+-------------+-----------+----+-----------------+--------------------+--------------------+-------------------+
|marketplace|customer_id|     review_id|product_id|product_parent|       product_title|product_category|star_rating|helpful_votes|total_votes|vine|verified_purchase|     review_headline|         review_body|        review_date|
+-----------+-----------+--------------+----------+--------------+--------------------+----------------+-----------+-------------+-----------+----+-----------------+--------------------+--------------------+-------------------+
|         US|   36075342| RAB23OVFNCXZQ|B00LPRXQ4Y|     339193102|17" 2003-2006 For...|      Automotive|          1|            0|          0|   N|                Y|     As it was used,|As it was used, t...|2015-01-31 00:08:00|
|         US|   42462164|R3NORADVJO6IE6|B000C7S0TO|     907684644|Spectra Premium C...| 

## Drop duplicates and incomplete rows

In [5]:
print(df.count())
df = df.dropna()
print(df.count())
df = df.dropDuplicates()
print(df.count())

3514942
3514045
3514045


## Examine the schema

In [6]:
df.printSchema()

root
 |-- marketplace: string (nullable = true)
 |-- customer_id: integer (nullable = true)
 |-- review_id: string (nullable = true)
 |-- product_id: string (nullable = true)
 |-- product_parent: integer (nullable = true)
 |-- product_title: string (nullable = true)
 |-- product_category: string (nullable = true)
 |-- star_rating: integer (nullable = true)
 |-- helpful_votes: integer (nullable = true)
 |-- total_votes: integer (nullable = true)
 |-- vine: string (nullable = true)
 |-- verified_purchase: string (nullable = true)
 |-- review_headline: string (nullable = true)
 |-- review_body: string (nullable = true)
 |-- review_date: timestamp (nullable = true)



## Create a new DataFrames for desired info

In [7]:
review_id_table = df.select(["review_id", "customer_id", "product_id", "product_parent", "review_date"])
review_id_table.show(10)

+--------------+-----------+----------+--------------+-------------------+
|     review_id|customer_id|product_id|product_parent|        review_date|
+--------------+-----------+----------+--------------+-------------------+
|R1007S364J8CDQ|   42693892|B002ZTM82W|     223407117|2013-01-30 00:04:00|
|R100F62R1FKX6K|   27035803|B000C7YQ9M|     411550379|2015-01-22 00:07:00|
|R100P5O4H4UT77|    6948080|B000C581WM|     932716999|2014-01-29 00:10:00|
|R100RPZTD25VKA|   37916687|B007DG3LB4|     695816221|2015-01-13 00:03:00|
| R100W8U0H2RQG|   14521836|B004QMHGVO|     566077034|2014-01-27 00:05:00|
| R101G73SA0RWE|   23102910|B003UUDD4C|     445144208|2015-01-14 00:04:00|
|R101UJWDOG0IKY|   11729151|B005XUESPK|     619961973|2013-01-28 00:05:00|
|R102GU8JMWXTYZ|   40308767|B009ATYQZC|     598680620|2015-01-03 00:03:00|
|R102LI1FGXOHWR|   34825748|B002NZJEHK|     921792604|2014-01-21 00:11:00|
|R102OXXLFH53SA|   14122330|B005DKIIJW|      21630215|2012-01-22 00:08:00|
+--------------+---------

In [8]:
#This table will contain only unique values
products = df.select(["product_id", "product_title"])
products.show(10)

+----------+--------------------+
|product_id|       product_title|
+----------+--------------------+
|B002ZTM82W|Raider 33-0584 Sp...|
|B000C7YQ9M|Stant 10840 Fuel Cap|
|B000C581WM|Moog SB373 Leaf S...|
|B007DG3LB4|DUMBO ELEPHANT Bl...|
|B004QMHGVO|RaceQuip SFI 16.1...|
|B003UUDD4C|aFe 31-10196 Magn...|
|B005XUESPK|ROADPRO LED Licen...|
|B009ATYQZC|Freedom 52535 Tri...|
|B002NZJEHK|31oz. Finish Kare...|
|B005DKIIJW|Stanley Battery M...|
+----------+--------------------+
only showing top 10 rows



In [16]:
# Drop duplicates due to some ids and names repeating
products= products.dropDuplicates()
products.count()
products.show(5)

+----------+--------------------+
|product_id|       product_title|
+----------+--------------------+
|B0088LL7UA|RAM Mounts (RAM-H...|
|B009YKQR2C|Zone Tech 4pcs 20...|
|B0089GTC92|Classy Autos Back...|
|B00063Y156|OTC 4533 Stinger ...|
|B00075XFDM|Go Rhino! (3093C)...|
+----------+--------------------+
only showing top 5 rows



In [9]:
customers = df.select(["customer_id"])
customers.show(10)

+-----------+
|customer_id|
+-----------+
|   42693892|
|   27035803|
|    6948080|
|   37916687|
|   14521836|
|   23102910|
|   11729151|
|   40308767|
|   34825748|
|   14122330|
+-----------+
only showing top 10 rows



In [10]:
customers = customers.groupby(['customer_id']).count()
customers.show(10)

+-----------+-----+
|customer_id|count|
+-----------+-----+
|   11914820|    3|
|   26872164|   53|
|   51355483|   13|
|    7232956|    1|
|   46357026|    3|
|   12705337|    1|
|   43997831|   13|
|   20903438|    5|
|   29485007|   12|
|   30377250|    1|
+-----------+-----+
only showing top 10 rows



In [11]:
customers = customers.withColumnRenamed("count", "customer_count")
customers.show(5)

+-----------+--------------+
|customer_id|customer_count|
+-----------+--------------+
|   11914820|             3|
|   26872164|            53|
|   51355483|            13|
|    7232956|             1|
|   46357026|             3|
+-----------+--------------+
only showing top 5 rows



In [24]:
vine_table = df.select(["review_id", "star_rating", "helpful_votes", "total_votes", "vine"])
vine_table.show(10)

+--------------+-----------+-------------+-----------+
|     review_id|star_rating|helpful_votes|total_votes|
+--------------+-----------+-------------+-----------+
|R1007S364J8CDQ|          3|            0|          0|
|R100F62R1FKX6K|          5|            0|          0|
|R100P5O4H4UT77|          5|            0|          0|
|R100RPZTD25VKA|          5|            0|          0|
| R100W8U0H2RQG|          5|            0|          0|
| R101G73SA0RWE|          4|            3|          3|
|R101UJWDOG0IKY|          4|            3|          3|
|R102GU8JMWXTYZ|          5|            0|          0|
|R102LI1FGXOHWR|          5|            1|          1|
|R102OXXLFH53SA|          1|           70|         87|
+--------------+-----------+-------------+-----------+
only showing top 10 rows



## Write DataFrame to RDS

In [20]:
# Configuration for RDS instance
mode="append"
jdbc_url = "jdbc:postgresql://<end Point here>:<port>/<db_name_here>"
config = {"user":"root",
          "password": "<your Password Here>",
          "driver":"org.postgresql.Driver"}

## Write DataFrame to RDS

In [14]:
review_id_table.write.jdbc(url=jdbc_url, table='review_id_table', mode=mode, properties=config)

In [18]:
products.write.jdbc(url=jdbc_url, table='products', mode=mode, properties=config)

In [23]:
customers.write.jdbc(url=jdbc_url, table='customers', mode=mode, properties=config)

In [25]:
vine_table.write.jdbc(url=jdbc_url, table='vine_table', mode=mode, properties=config)